<a href="https://colab.research.google.com/github/Kumnoob/Human-Identification-from-Text-Messages/blob/main/DataToCSV/data_cleaning_enron_email_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import necessary libraries

In [3]:
import multiprocessing
import seaborn as sns
import email
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'seaborn'

### Load Data

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/Users/USER/Desktop/project/emails.csv")

### Data Exploration

In [ ]:
# view first 5 rows of the dataset
df.head()

In [ ]:
# get shape of the data
df.shape

In [ ]:
# a sample email
print(df.loc[1]['message'])

In [ ]:
# transform the email into correct format
message = df.loc[1]['message']
e = email.message_from_string(message)

e.items()

In [ ]:
# get date
e.get('Date')

In [ ]:
# show message body
e.get_payload()

### Extract headers

In [ ]:
# now we add those fields into our 'df' dataframe
def get_field(field, messages):
    column = []
    for message in messages:
        e = email.message_from_string(message)
        column.append(e.get(field))
    return column

In [ ]:
df['date'] = get_field("Date", df['message'])
df['subject'] = get_field("Subject", df['message'])
df['X-Folder'] = get_field("X-Folder", df['message'])
df['X-From'] = get_field("X-From", df['message'])
df['X-To'] = get_field("X-To", df['message'])
df.head(3)

### Extract Message Body

In [ ]:
def body(messages):
    column = []
    for message in messages:
        e = email.message_from_string(message)
        column.append(e.get_payload())
    return column

df['body'] = body(df['message'])
df.head(3)

### Employee Names

In [ ]:
df['file'][:10]

In [ ]:
def employee(file):
    column = []
    for string in file:
        column.append(string.split("/")[0])
    return column

df['employee'] = employee(df['file'])
df.head(3)

### Look into X-Folder

In [ ]:
print("number of folders: ", df.shape[0])
print("number of unique folders: ", df['X-Folder'].unique().shape[0])

In [ ]:
unique_emails = pd.DataFrame(df['X-Folder'].value_counts())
unique_emails.reset_index(inplace=True)


unique_emails.columns = ['folder_name', 'count']
# top 20 folders
unique_emails.iloc[:20,:]

### Visualize top 20 folders

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x='count', y='folder_name', data=unique_emails.iloc[:20, :], palette="Blues_d")
plt.title("Top 20 folders")
plt.xlabel("Count")
plt.ylabel("Folder_Name")
plt.show()

**Let's see top 20 highest email sender employee**

In [ ]:
top_20 = pd.DataFrame(df['employee'].value_counts()[:20])
top_20.reset_index(inplace=True)
top_20.columns = ["Employee_name", "Counts"]
top_20

**Visualize top 20 highest email sender employees**

In [ ]:
plt.figure(figsize=(10,8))

sns.barplot(y="Employee_name", x="Counts", data=top_20, palette="Blues_d")
plt.title("Top 20 highest email sender employee")
plt.xlabel("Count")
plt.ylabel("Employee_name")
plt.show()

## Data Cleaning and Transformation

**date : column**

In [ ]:
import datetime
from dateutil import parser

# this is sample example
x = parser.parse("Fri, 4 May 2001 13:51:00 -0700 (PDT)")
print(x.strftime("%d-%m-%Y %H:%M:%S"))

In [ ]:
def change_type(dates):
    column = []
    
    for date in dates:
        column.append(parser.parse(date).strftime("%d-%m-%Y %H:%M:%S"))
    return column

df['date'] = change_type(df['date'])
df.head(2)

**Column : X-Folder**

In [ ]:
print(df['X-Folder'][0])

# we only want last folder name
df['X-Folder'][0].split("\\")[-1]

In [ ]:
def preprocess_folder(folders):
    column = []
    for folder in folders:
        if (folder is None or folder == ""):
            column.append(np.nan)
        else:
            column.append(folder.split("\\")[-1].lower())
    return column

df['X-Folder'] = preprocess_folder(df['X-Folder'])
df.head(2)

In [ ]:
# count unique folders
print("Unique Foldes: ", len(df['X-Folder'].unique()))

# view some of them
df['X-Folder'].unique()[0:20]

**Replace empty missing values in subject with np.nan**

In [ ]:
def replace_empty_with_nan(subject):
    column = []
    for val in subject:
        if (val == ""):
            column.append(np.nan) 
        else:
            column.append(val)
    return column

In [ ]:
df['subject'] = replace_empty_with_nan(df['subject'])
df['X-To'] = replace_empty_with_nan(df['X-To'])

In [ ]:
df.isnull().sum()

In [ ]:
# calculate percentage of missing values
miss = df.isnull().sum()
miss = miss[miss>0]
miss = miss / df.shape[0]
miss

In [ ]:
# drop missing value rows
df.dropna(axis=0, inplace=True)

In [ ]:
df.isnull().sum(), df.shape

In [ ]:
df.head(3)

Drop the following columns:
- file
- message
- date
- X-From
- X-To

In [ ]:
cols_to_drop = ['file','message','date', 'X-From', 'X-To']

In [ ]:
df.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df

In [ ]:
df = df[df.employee != "arora-h"][df.employee != "badeer-r"][df.employee != "bailey-s"][df.employee != "benson-r"][df.employee != "causholli-m"][df.employee != "crandell-s"][df.employee != "dickson-s"][df.employee != "forney-j"][df.employee != "gang-l"][df.employee != "gilbertsmith-d"][df.employee != "harris-s"][df.employee != "hendrickson-s"][df.employee != "holst-k"][df.employee != "king-j"][df.employee != "lucci-p"][df.employee != "mccarty-d"][df.employee != "mckay-b"][df.employee != "mckay-j"][df.employee != "motley-m"][df.employee != "panus-s"][df.employee != "pereira-s"][df.employee != "phanis-s"][df.employee != "pimenov-v"][df.employee != "platter-p"][df.employee != "quenet-j"][df.employee != "rapp-b"][df.employee != "reitmeyer-j"][df.employee != "richey-c"][df.employee != "ring-a"][df.employee != "ring-r"][df.employee != "sanchez-m"][df.employee != "scholtes-d"][df.employee != "schwieger-j"][df.employee != "semperger-c"][df.employee != "slinger-r"][df.employee != "south-s"][df.employee != "staab-t"][df.employee != "swerzbin-m"][df.employee != "townsend-j"][df.employee != "whitt-m"][df.employee != "zufferli-j"]

In [ ]:
df

In [ ]:
# save the data
df.to_csv("cleaned_data.csv", index=False)

# Doc2Vec

In [1]:
import csv
import sys
maxInt = sys.maxsize

while True:
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

import numpy as np 
import pandas as pd 
import gensim
import gensim.downloader as api
from gensim.models.doc2vec import TaggedDocument, Doc2Vec 
from gensim.test.utils import get_tmpfile

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import utils

import random

from tqdm import tqdm
import multiprocessing
import nltk
from nltk.corpus import stopwords


In [2]:
df = pd.read_csv("/Users/USER/Desktop/project/cleaned_data.csv")

In [3]:
df 

,subject,X-Folder,body,employee
0,Re:,'sent mail,Traveling to have a business meeting takes the...,allen-p
1,Re: test,'sent mail,test successful. way to go!!!,allen-p
2,Re: Hello,'sent mail,Let's shoot for Tuesday at 11:45.,allen-p
3,Re: Hello,'sent mail,"Greg,\n\n How about either next Tuesday or Thu...",allen-p
4,Re: PRC review - phone calls,'sent mail,any morning between 10 and 11:30,allen-p
...,...,...,...,...
465935,Accenture Deal Points,tss,Attached is the document that will serve as th...,zipper-a
465936,Licensing Agreement,tss,"\nAt the request of Travis McCullough, I am f...",zipper-a
465937,Licensing Agreement,tss,Attached for your review is a revised draft of...,zipper-a
465938,RE: Accenture Agreement,tss,Sorry for the delay in responding. I needed t...,zipper-a


In [3]:
employee = df['employee']
def dupli(x):
    return list(dict.fromkeys(x))
employee = dupli(employee)

In [4]:
employee

['allen-p',
 'arnold-j',
 'bass-e',
 'baughman-d',
 'beck-s',
 'blair-l',
 'brawner-s',
 'buy-r',
 'campbell-l',
 'carson-m',
 'cash-m',
 'corman-s',
 'cuilla-m',
 'dasovich-j',
 'davis-d',
 'dean-c',
 'delainey-d',
 'derrick-j',
 'donoho-l',
 'donohoe-t',
 'dorland-c',
 'ermis-f',
 'farmer-d',
 'fischer-m',
 'fossum-d',
 'gay-r',
 'geaccone-t',
 'germany-c',
 'giron-d',
 'griffith-j',
 'grigsby-m',
 'guzman-m',
 'haedicke-m',
 'hain-m',
 'hayslett-r',
 'heard-m',
 'hernandez-j',
 'hodge-j',
 'horton-s',
 'hyatt-k',
 'hyvl-d',
 'jones-t',
 'kaminski-v',
 'kean-s',
 'keavey-p',
 'keiser-k',
 'kitchen-l',
 'kuykendall-t',
 'lavorato-j',
 'lay-k',
 'lenhart-m',
 'lewis-a',
 'linder-e',
 'lokay-m',
 'lokey-t',
 'love-p',
 'maggi-m',
 'mann-k',
 'martin-t',
 'may-l',
 'mcconnell-m',
 'mclaughlin-e',
 'merriss-s',
 'meyers-a',
 'mims-thurston-p',
 'neal-s',
 'nemec-g',
 'parks-j',
 'perlingiere-d',
 'presto-k',
 'quigley-d',
 'rodrique-r',
 'rogers-b',
 'ruscitti-k',
 'sager-e',
 'saibi-e',


In [4]:
employee_index = {}
index_count = 1
for i in employee:
    employee_index[i] = index_count
    index_count += 1

In [5]:
employee_index

{'allen-p': 1,
 'arnold-j': 2,
 'bass-e': 3,
 'baughman-d': 4,
 'beck-s': 5,
 'blair-l': 6,
 'brawner-s': 7,
 'buy-r': 8,
 'campbell-l': 9,
 'carson-m': 10,
 'cash-m': 11,
 'corman-s': 12,
 'cuilla-m': 13,
 'dasovich-j': 14,
 'davis-d': 15,
 'dean-c': 16,
 'delainey-d': 17,
 'derrick-j': 18,
 'donoho-l': 19,
 'donohoe-t': 20,
 'dorland-c': 21,
 'ermis-f': 22,
 'farmer-d': 23,
 'fischer-m': 24,
 'fossum-d': 25,
 'gay-r': 26,
 'geaccone-t': 27,
 'germany-c': 28,
 'giron-d': 29,
 'griffith-j': 30,
 'grigsby-m': 31,
 'guzman-m': 32,
 'haedicke-m': 33,
 'hain-m': 34,
 'hayslett-r': 35,
 'heard-m': 36,
 'hernandez-j': 37,
 'hodge-j': 38,
 'horton-s': 39,
 'hyatt-k': 40,
 'hyvl-d': 41,
 'jones-t': 42,
 'kaminski-v': 43,
 'kean-s': 44,
 'keavey-p': 45,
 'keiser-k': 46,
 'kitchen-l': 47,
 'kuykendall-t': 48,
 'lavorato-j': 49,
 'lay-k': 50,
 'lenhart-m': 51,
 'lewis-a': 52,
 'linder-e': 53,
 'lokay-m': 54,
 'lokey-t': 55,
 'love-p': 56,
 'maggi-m': 57,
 'mann-k': 58,
 'martin-t': 59,
 'may-l': 

In [6]:
documents = df['body']

In [8]:
documents

0         Traveling to have a business meeting takes the...
1                            test successful.  way to go!!!
2                       Let's shoot for Tuesday at 11:45.  
3         Greg,\n\n How about either next Tuesday or Thu...
4                          any morning between 10 and 11:30
                                ...                        
465935    Attached is the document that will serve as th...
465936    \nAt the request of  Travis McCullough, I am f...
465937    Attached for your review is a revised draft of...
465938    Sorry for the delay in responding.  I needed t...
465939    Resending......I think I screwed up the first ...
Name: body, Length: 465940, dtype: object

In [7]:
data = [d for d in documents]

In [ ]:
data

In [8]:
print(len(data))

465940


In [9]:
data[0]

"Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.\n\nAs far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the presenter speaks and the others are quiet just waiting for their turn.   The meetings might be better if held in a round table discussion format.  \n\nMy suggestion for where to go is Austin.  Play golf and rent a ski boat and jet ski's.  Flying somewhere takes too much time.\n"

In [24]:
data[1]

'test successful.  way to go!!!'

In [22]:
tqdm.pandas(desc="progress-bar")
# Function for tokenizing
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
# Initializing the variables
train_documents = []
test_documents = []
i = 0
# Associating the tags(labels) with numbers
#tags_index = {'sci-fi': 1 , 'action': 2, 'comedy': 3, 'fantasy': 4, 'animation': 5, 'romance': 6}
#Reading the file
FILEPATH = '/Users/USER/Desktop/project/cleaned_data.csv'
with open(FILEPATH, 'r') as csvfile:
    with open('/Users/USER/Desktop/project/cleaned_data.csv', 'r') as csvfile:
        body = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in body:
            if i == 0:
                i+=1
            else:
                train_documents.append(TaggedDocument(words=tokenize_text(row[2]), tags=[employee_index.get(row[3], 8)] ))


In [23]:
train_documents[1].tags[0]

1

In [13]:
for i in train_documents:
    if i.tags[0] != 1:
        test_documents.append(i)


In [13]:
len(test_documents)

463768

In [24]:
train1 = []
train2 = []

In [25]:
hold_temp = []
hold_temp = train_documents

In [26]:
hold_temp[1]

TaggedDocument(words=['test', 'successful', 'way', 'to', 'go'], tags=[1])

In [27]:
train_documents = []
for i in hold_temp:
    if i.tags[0] == 1:
        train1.append(i)

In [28]:
for i in hold_temp:
    if i.tags[0] == 2:
        train2.append(i)

In [33]:
len(train1)

1672

In [34]:
len(train2)

2827

In [31]:
for i in range(500):
    test_documents.append(train1.pop(random.randrange(len(train1))))
for i in range(1000):
    test_documents.append(train2.pop(random.randrange(len(train2))))

In [17]:
final_test_document = []
for i in range(300):
    final_test_document.append(train_documents.pop(random.randrange(len(train_documents))))
for i in range(300):
    final_test_document.append(test_documents.pop(random.randrange(len(test_documents))))

In [32]:
len(test_documents)

1500

In [35]:
train_documents = train1 + train2

In [36]:
len(train_documents)

4499

In [37]:
cores = multiprocessing.cpu_count()

model_dbow = Doc2Vec(dm=1, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores, alpha=0.025, min_alpha=0.001)
model_dbow.build_vocab([x for x in tqdm(train_documents)])
train_documents  = utils.shuffle(train_documents)
model_dbow.train(train_documents,total_examples=len(train_documents), epochs=30)


100%|██████████| 4499/4499 [00:00<00:00, 1503479.70it/s]


In [38]:
def vector_for_learning(model, input_docs):
    sents = input_docs
    targets, feature_vectors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, feature_vectors

In [40]:
y_train, X_train = vector_for_learning(model_dbow, train_documents)
y_test, X_test = vector_for_learning(model_dbow, test_documents)

logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy0.8846666666666667
Testing F1 score: 0.8856372626800568


c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
nltk.download('punkt')
all_data = []
for sentence in data:
  one_data = nltk.word_tokenize(sentence)
  all_data.append(one_data)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
def tagged_document(list_of_list_of_words):
  for i, list_of_words in enumerate(list_of_list_of_words):
    yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])

data_for_training = list(tagged_document(all_data))
# print(data_for_training[:100])

model = gensim.models.doc2vec.Doc2Vec(vector_size = 300, min_count=2, epochs = 10)
model.build_vocab(data_for_training)

In [ ]:
model.train(data_for_training, total_examples=model.corpus_count, epochs=model.epochs)

In [26]:
data_for_training[3]

TaggedDocument(words=['Greg', ',', 'How', 'about', 'either', 'next', 'Tuesday', 'or', 'Thursday', '?', 'Phillip'], tags=[3])

In [15]:
model.docvecs[462738]

C:\Users\USER\AppData\Local\Temp\ipykernel_26728\4114686976.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  model.docvecs[462738]


array([-2.07089204e-02, -5.45915842e-01, -1.16873562e-01, -2.10034162e-01,
        7.95039311e-02,  2.65169758e-02, -5.76359183e-02,  1.80075526e-01,
        5.67163415e-02, -9.97347534e-02,  1.01480938e-01, -9.94654968e-02,
        6.72458947e-01,  5.13300776e-01, -5.04152298e-01,  5.94831072e-02,
        3.13416839e-01, -1.79293811e-01, -2.80299127e-01,  1.62524104e-01,
        3.45085450e-02, -4.31472033e-01,  8.45711753e-02,  5.19633532e-01,
       -1.15027085e-01, -9.65711474e-02,  4.14995164e-01,  4.31038827e-01,
       -2.27732956e-01, -4.55580771e-01,  2.93609887e-01, -6.42205551e-02,
        2.56475568e-01, -1.02794103e-01, -3.63340825e-01, -4.27139580e-01,
        1.81274280e-01, -7.46911857e-03, -2.54485309e-01, -7.01408684e-02,
        4.11859393e-01,  2.84865052e-01,  2.87494749e-01, -5.33745885e-01,
        3.93379003e-01,  1.40850261e-01,  3.68354656e-02,  3.62937331e-01,
        3.11967395e-02,  3.43228072e-01,  3.65518302e-01,  3.04722160e-01,
       -1.46203384e-01, -

In [16]:
model.docvecs.most_similar(0)

C:\Users\USER\AppData\Local\Temp\ipykernel_26728\3754858182.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  model.docvecs.most_similar(0)


[(634, 0.9208298325538635),
 (203, 0.9089654088020325),
 (1739, 0.8986125588417053),
 (1954, 0.8816961646080017),
 (239470, 0.8375118970870972),
 (334568, 0.6670061945915222),
 (428356, 0.6592144966125488),
 (448481, 0.6533142924308777),
 (433300, 0.6527002453804016),
 (424653, 0.6518880724906921)]

In [24]:
model.save("/vscode/finalProject/Human-Identification-from-Text-Messages/DataToCSV/doc2vec.h5")

In [14]:
model = Doc2Vec.load("/vscode/finalProject/Human-Identification-from-Text-Messages/DataToCSV/doc2vec.h5")